# Second set for Non-Experts

We had about 6000 fields labeled by non-experts.
Some of them were sent to Experts!

We want to do it a second time. Here,

  - The fields are limited to a particular set of crops, mostly.
  - Are chosen from the fields that are not sent to Experts.

In [1]:
import pandas as pd
import csv
import numpy as np
import os, os.path
import sys

import collections # to count frequency of elements of an array
# to move files from one directory to another
import shutil

In [2]:
param_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
choices_xl = pd.ExcelFile(param_dir + "all_extended.xlsx")
choices_sheet_names = choices_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(param_dir + "6000responses.xlsx")
response_sheet_names = response_set_1_xl.sheet_names  # see all sheet names

print (choices_sheet_names[:5])
print (response_sheet_names[:5])

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5']
['Set 1 E', 'Set 2 E', 'Set 3 E', 'Set 4 E', 'Set 5 E']


In [4]:
# evaluation_set_csv = pd.read_csv(param_dir + "evaluation_set.csv")
# evaluation_set_csv.drop(labels=["ExctAcr"], axis='columns', inplace=True)

# print (len(evaluation_set_csv.ID))
# print (len(evaluation_set_csv.ID.unique()))

# Read and assemble the choices; i.e. selected fields for survey

In [5]:
survey_fields = pd.DataFrame()
for a_sheet in choices_sheet_names:
    a_choice_sheet = choices_xl.parse(a_sheet)
    survey_fields = pd.concat([survey_fields, a_choice_sheet])

print (survey_fields.shape)
print (len(survey_fields.ID))
print (len(survey_fields.ID.unique()))

(6340, 17)
6340
6340


In [6]:
survey_fields.head(2)

,Question_in_set,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,1,1,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.832739,-118.912092,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams
1,2,2,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.750901,-118.979764,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,2,2.480750,2016/08/03 00:00:00,Adams


## Count number of questions

In [7]:
([x for x in response_sheet_names if 'problem' in x.lower()])

['Set 6 E - Problem',
 'Set 15 Problem',
 'Set 20 Problem',
 'Set 21 - Problem',
 'Set 27 Problem',
 'Set 29 Problem',
 'Set 31 Problem',
 'Set 32 Problem',
 'Set 38 Problem',
 'Set 41 Problem',
 'Set 42 Problem',
 'Set 48 Problem']

In [8]:
question_count = 0

for a_choice_sheet in choices_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [6340] questions.


# Clean Vote count start here:

#### Problems:
 - In the beginning emails were not collected! (mostly alfalfa and apple?)
 - There are some Forms that I still do not have access to. Eshwar has not transferred them to Google Drive despite repeated emails.
 - There are forms with repeated answers and no email.
 - Min has responded sparsley, mostly no email, and problematic forms. 
 - Different email for Hossein. Since I was receiving emails after completing forms, I used a fake email!! no, thank you!
 - Problematic forms were responded by some, and after fix by others. Drop these forms.
 - Not much Mike's vote in wheat. 

In [9]:
## Define the damn output dataframe
output_df = pd.DataFrame(columns=['Form', 'Question', 'ID',
                                  "Hossein", "Supriya", "Kirti", "Mike", "Min"], 
                                   index=range(question_count))
output_df.head(1)
curr_row = 0

extended_choices = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_sheet_names:
    # pick up the numeric part of the sheet names
    sheet_numeric_part = response_sheet_name.split()[1]
    
    # Form sheet names of choices excel sheets
    choice_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_xl.parse(choice_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)

    # If no email is recoded, pass. We do not want it
    # a_response_sheet['Email Address'].isnull().any() works as well. 
    # It is an indication that the form did not collect emails! One or all!
    if a_response_sheet['Email Address'].isnull().all():
        continue

    
    # Fix Hossein email. (replace emails by names!)
    a_response_sheet['Email Address'] = a_response_sheet['Email Address'].str.lower()

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('noorazar', na=False), 
                         'Email Address']="Hossein"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('kirti', na=False), 
                         'Email Address']="Kirti"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('supriya', na=False), 
                         'Email Address']="Supriya"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('brady', na=False), 
                         'Email Address']="Mike"

    a_response_sheet.loc[a_response_sheet['Email Address'].str.contains('ming', na=False), 
                         'Email Address']="Min"

    # Fix repeated Kirti in one sheet
    #
    #   IF Kirti responded to a problematic sheet, and then to fixed sheet,
    #   Then we have duplicates!!!!!
    #
    latest_Kirti = a_response_sheet[a_response_sheet['Email Address']=="Kirti"].Timestamp.max()
    Kirti = a_response_sheet[a_response_sheet['Email Address']=="Kirti"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Kirti[Kirti.Timestamp < latest_Kirti].index
    a_response_sheet.drop(bad_index, inplace=True)

    latest_Hossein = a_response_sheet[a_response_sheet['Email Address']=="Hossein"].Timestamp.max()
    Hossein = a_response_sheet[a_response_sheet['Email Address']=="Hossein"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Hossein[Hossein.Timestamp < latest_Hossein].index
    a_response_sheet.drop(bad_index, inplace=True)
    
    
    latest_Supriya = a_response_sheet[a_response_sheet['Email Address']=="Supriya"].Timestamp.max()
    Supriya = a_response_sheet[a_response_sheet['Email Address']=="Supriya"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Supriya[Supriya.Timestamp < latest_Supriya].index
    a_response_sheet.drop(bad_index, inplace=True)
    
    latest_Mike = a_response_sheet[a_response_sheet['Email Address']=="Mike"].Timestamp.max()
    Mike = a_response_sheet[a_response_sheet['Email Address']=="Mike"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Mike[Mike.Timestamp < latest_Mike].index
    a_response_sheet.drop(bad_index, inplace=True)
    
    latest_Min = a_response_sheet[a_response_sheet['Email Address']=="Min"].Timestamp.max()
    Min = a_response_sheet[a_response_sheet['Email Address']=="Min"]
    # The fact that we have "<" below, eliminates to count how many times
    # Kirti has responded. i.e. if there is one response, it will not be dropped!
    bad_index = Min[Min.Timestamp < latest_Min].index
    a_response_sheet.drop(bad_index, inplace=True)

    if len(a_response_sheet['Email Address']) != len(a_response_sheet['Email Address'].unique()):
        raise ValueError("Something is wrong in email address column")

    for a_col_name in a_response_sheet.columns:
        if "http" in a_col_name:
            question_number = a_col_name.split()[1].split(":")[0]
            currnt_ID = a_choice_sheet.loc[int(question_number)-1, "ID"]
            if currnt_ID in list(output_df.ID):
                curr_idx = output_df[output_df.ID == currnt_ID].index
                
                if "Hossein" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Hossein"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Hossein"][a_col_name].values[0]

                if "Supriya" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Supriya"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Supriya"][a_col_name].values[0]

                if "Kirti" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Kirti"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Kirti"][a_col_name].values[0]
                if "Mike" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Mike"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Mike"][a_col_name].values[0]
                
                if "Min" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_idx, "Min"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Min"][a_col_name].values[0]
            else:
                output_df.loc[curr_row, "ID"] = currnt_ID
                output_df.loc[curr_row, "Form"] = int(sheet_numeric_part)
                output_df.loc[curr_row, "Question"] = int(question_number)
                
                if "Hossein" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Hossein"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Hossein"][a_col_name].values[0]

                if "Supriya" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Supriya"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Supriya"][a_col_name].values[0]

                if "Kirti" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Kirti"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Kirti"][a_col_name].values[0]
                if "Mike" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Mike"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Mike"][a_col_name].values[0]
                
                if "Min" in list(a_response_sheet["Email Address"]):
                    output_df.loc[curr_row, "Min"]=a_response_sheet[a_response_sheet["Email Address"] == \
                                                                     "Min"][a_col_name].values[0]
                curr_row += 1

output_df.dropna(how='all', axis='rows', inplace=True);
print (output_df.shape)
print (len(output_df.ID))
print (len(output_df.ID.unique()))

(5740, 8)
5740
5740


In [10]:
output_df.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min
0,1,1,104119_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop
1,1,2,51403_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Unsure


# Correct the damn responses

1. We have not been consistent in the Forms!
2. The options we gave there was horrible. For example, the options were:
   - Single Crop
   - Double Crop
   - Unsure
   - Other:
   
Convert anything that has ```cover```, ```mustard``` in it to ```double-crop```, single crops are single crops, anything else will be ```Unsure```.

If an answer has any of these pairs in it, it will be labeled as unsure ```[single, double]```, ```[single, cover]```,
```[single, mustard]```.

In [11]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop',
       'May be mustard growing here second curve from aug-sept onwards',
       'may be mustard',
       'May be non-mustard cover crop suggested by second curve',
       'Looks like a non-mustard cover crop', 'Double Crop/Cover crop',
       nan, 'Unsure'], dtype=object)

In [12]:
output_df.reset_index(inplace=True, drop=True)
output_backup = output_df.copy()

In [13]:
output_df = output_backup.copy()

In [14]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop',
       'May be mustard growing here second curve from aug-sept onwards',
       'may be mustard',
       'May be non-mustard cover crop suggested by second curve',
       'Looks like a non-mustard cover crop', 'Double Crop/Cover crop',
       nan, 'Unsure'], dtype=object)

In [15]:
people = ["Hossein", "Supriya", "Kirti", "Mike", "Min"]

# We need to keep "none" so that we can use it to compute
# majority vote where someone has not responded a Form
output_df[people] = output_df[people].fillna('none')

for idx in output_df.index:
    for person in people:
        if output_df.loc[idx, person] in ["Single Crop", "Double Crop", "Unsure", "none"]:
            continue
            
        if "cover" in output_df.loc[idx, person].lower() or "mustard" in output_df.loc[idx, person].lower():
            output_df.loc[idx, person] = "Double Crop"
        
        else:
            output_df.loc[idx, person] = "Unsure"
            
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop', 'none', 'Unsure'], dtype=object)

In [16]:
len(output_df.ID.unique())

5740

In [17]:
output_df.Supriya.unique()

array(['Single Crop', 'Double Crop', 'none', 'Unsure'], dtype=object)

In [18]:
survey_fields.head(2)

,Question_in_set,Question_overall,ID,NDVI_TS_Name,corrected_RGB,TOA_RGB,latitude,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,1,1,104119_WSDA_SF_2016,AdamBenton2016_104119_WSDA_SF_2016.png,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,104119_WSDA_SF_2016_46.83273927_-118.91209151_...,46.832739,-118.912092,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams
1,2,2,51403_WSDA_SF_2016,AdamBenton2016_51403_WSDA_SF_2016.png,51403_WSDA_SF_2016_46.75090118_-118.9797641_co...,51403_WSDA_SF_2016_46.75090118_-118.9797641_TO...,46.750901,-118.979764,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,2,2.480750,2016/08/03 00:00:00,Adams


In [19]:
extended_output = pd.merge(output_df, survey_fields, on=['ID'], how='left')
extended_output.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Question_in_set,Question_overall,...,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,1,1,104119_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,1,1,...,-118.912092,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams
1,1,2,51403_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Unsure,2,2,...,-118.979764,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,2,2.480750,2016/08/03 00:00:00,Adams


In [20]:
len(extended_output.ID.unique())

5740

# Drop the fields that are labeled by Experts

#### Read First set

In [21]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
set1_fields = pd.read_csv(perry_dir + "set_1_experts_stats_extended_sortOpinionCrop.csv")
print (set1_fields.shape)
set1_fields.head(2)

(294, 21)


,Form,Question,opinion_count,ID,Perry,Andrew,Tim,Question_in_set,Question_overall,NDVI_TS_Name,...,TOA_RGB,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,3,32,3,99837_WSDA_SF_2017,Double Crop,Single Crop,Mustard Crop,32,132,Grant2017_99837_WSDA_SF_2017.png,...,99837_WSDA_SF_2017_46.87015998_-119.40630986_T...,46.87016,-119.406310,barley,center pivot,wsda,73,73.250981,2017/10/10,Grant
1,2,22,3,98359_WSDA_SF_2017,Double Crop,Unsure,Single Crop,22,72,Grant2017_98359_WSDA_SF_2017.png,...,98359_WSDA_SF_2017_46.93911002_-119.65673394_T...,46.93911,-119.656734,"bean, dry",rill,wsda,24,24.163431,2017/07/10,Grant


#### Read Second Set

In [22]:
param_in_data_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
hand_picked_set2 = pd.read_csv(param_in_data_dir + "threeHundred_IDs_Set2_Perry.csv")
hand_picked_set2.dropna(inplace=True)
print (hand_picked_set2.shape)
hand_picked_set2.head(2)

(322, 1)


,ID
0,52701_WSDA_SF_2016
1,53739_WSDA_SF_2016


#### Get unique IDs of first and second set

In [23]:
expert_IDs = list(hand_picked_set2.ID) + list(set1_fields.ID)
expert_IDs = list(set(expert_IDs))
len(expert_IDs)
print (len(extended_output.ID.unique()))
print (len(extended_output.ID))

5740
5740


In [24]:
extended_output.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Question_in_set,Question_overall,...,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,1,1,104119_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Single Crop,1,1,...,-118.912092,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,26,26.413875,2016/08/17 00:00:00,Adams
1,1,2,51403_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,Single Crop,Unsure,2,2,...,-118.979764,How would you label this field?,Notes if you want to add any.,alfalfa hay,center pivot,wsda,2,2.480750,2016/08/03 00:00:00,Adams


## Drop the fields that are labeled by Experts

In [25]:
nonExpert_extended_output = extended_output[~extended_output.ID.isin(expert_IDs)]
len(nonExpert_extended_output.ID.unique())

5286

## Keep only limited crops

In [26]:
wanted_crops = ['alfalfa seed',
                'barley', 'barley hay', 'bean, dry', 'bean, green', 'bluegrass seed', 'buckwheat', 
                'canola', 'carrot', 'corn seed', 'corn, field', 'corn, sweet', 
                'grass hay', 'grass seed',
                'market crops', 'mint',
                'oat hay', 'onion',
                'pea seed', 'pea, dry', 'pea, green', 'potato', 
                'triticale',
                'wheat',
                'yellow mustard']

limitCrops_nonExpert_extended = nonExpert_extended_output[nonExpert_extended_output.CropTyp.isin(wanted_crops)]
limitCrops_nonExpert_extended.shape

(1429, 24)

In [27]:
len(limitCrops_nonExpert_extended.ID.unique())

1429

In [28]:
limitCrops_nonExpert_extended.head(2)

,Form,Question,ID,Hossein,Supriya,Kirti,Mike,Min,Question_in_set,Question_overall,...,longitude,Question Text,Text box Text,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
1202,31,3,53189_WSDA_SF_2016,Single Crop,Single Crop,Single Crop,none,none,3,1803,...,-118.573592,How would you label this field?,Notes if you want to add any.,barley,center pivot,wsda,132,131.696925,2016/09/20 00:00:00,Adams
1203,31,4,101534_WSDA_SF_2018,Single Crop,Double Crop,Single Crop,none,none,4,1804,...,-119.050357,How would you label this field?,Notes if you want to add any.,barley,center pivot,wsda,3,3.220729,2018/08/14 00:00:00,Franklin


In [90]:
nonExpert_survey2_fields = survey_fields[survey_fields.ID.isin(list(limitCrops_nonExpert_extended.ID))].copy()

nonExpert_survey2_fields.sort_values(by=['CropTyp', 'ID'], inplace=True)

nonExpert_survey2_fields.reset_index(inplace=True, drop=True)
nonExpert_survey2_fields.Question_in_set = 666
nonExpert_survey2_fields.Question_overall = 666
nonExpert_survey2_fields.shape
len(nonExpert_survey2_fields.ID.unique())

1429

# Write to Disc

In [91]:
# Be consistent with previous ones, so that the Google Scrip works
needed_cols = ["ID", 
               "NDVI_TS_Name", "corrected_RGB", "TOA_RGB", 
               "latitude", "longitude", "Question Text", "CropTyp", "Irrigtn", 
               "DataSrc", "Acres", "ExctAcr",
               "LstSrvD", "county"]

nonExpert_survey2_fields = nonExpert_survey2_fields[needed_cols]

In [92]:
out_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/nonExpert_set2_fields/"
out_name = out_dir + "nonExpert_survey2_fields.csv"
nonExpert_survey2_fields.to_csv(out_name, index = False)


In [93]:
no_questions = 60

if nonExpert_survey2_fields.shape[0] % no_questions != 0:
    no_dfs = nonExpert_survey2_fields.shape[0] // no_questions + 1
else:
    no_dfs = nonExpert_survey2_fields.shape[0] // no_questions

In [94]:
writer_extended = pd.ExcelWriter(out_dir + 'nonExpert_survey2_fields.xlsx', engine='xlsxwriter')

for ii in range(no_dfs):
    curr_result = nonExpert_survey2_fields.loc[(ii*no_questions): ((ii+1) * no_questions) - 1, ]
    curr_result.reset_index(drop=True, inplace=True)
    curr_result.to_excel(writer_extended, sheet_name= "NE_S2_F" + str(ii+1), index=False)

writer_extended.save()

In [95]:
choices_xl = pd.ExcelFile(out_dir + "nonExpert_survey2_fields.xlsx")
sheet_names = choices_xl.sheet_names  # see all sheet names


survey_fields_2_check = pd.DataFrame()
for a_sheet in sheet_names:
    a_choice_sheet = choices_xl.parse(a_sheet)
    survey_fields_2_check = pd.concat([survey_fields_2_check, a_choice_sheet])

print (survey_fields_2_check.shape)
print (len(survey_fields_2_check.ID))
print (len(survey_fields_2_check.ID.unique()))

survey_fields_2_check.sort_values(by=['CropTyp', 'ID'], inplace=True)
survey_fields_2_check.reset_index(inplace=True, drop=True)

survey_fields_2_check.equals(nonExpert_survey2_fields)

(1429, 14)
1429
1429


True

In [96]:
len(sheet_names)

24

In [97]:
nonExpert_survey2_fields.columns

Index(['ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude',
       'longitude', 'Question Text', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres',
       'ExctAcr', 'LstSrvD', 'county'],
      dtype='object')